In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install deep_autoviml

In [ ]:
from deep_autoviml import deep_autoviml as deepauto

In [ ]:
trainfile = '/kaggle/input/tabular-playground-series-dec-2021/train.csv'
testfile = '/kaggle/input/tabular-playground-series-dec-2021/test.csv'
subfile = '/kaggle/input/tabular-playground-series-dec-2021/sample_submission.csv'

In [ ]:
train = pd.read_csv(trainfile)
test = pd.read_csv(testfile)
sub = pd.read_csv(subfile)
print(train.shape, test.shape)
train.head()

In [ ]:
#### Check the class counts. If any class is too small, just drop those classes
target = 'Cover_Type'
train[target].value_counts()

In [ ]:
### Drop a couple of classes that dont have too many examples
train =  train[~(train[target].isin([4,5]))].sample(100000)
print(train.shape)

## Settting these options will enable us to try different speeds of model training:
keras_model_type settings are:
1. fast: a quick model that uses only dense layers (deep layers).
1. fast1: a deep and wide model that uses both deep and wide layers.
1. fast2: a deep and cross model that crosses some variables (hence deep and cross)
1. auto: This will try out multiple combinations of dense layers and optimize them using either Optuna or Storm-Tuner. This will take the longest time.

In [ ]:
keras_model_type = "fast" ### fast1 is a deep and wide model ###
project_name="Dec_TPS"
model_options = {'nlp_char_limit':10, 'cat_feat_cross_flag':False,
                 'max_trials': 10, 'early_stopping': True}
keras_options = {"patience":10, "epochs":100, 'early_stopping': True}

## Let's train the model using either the dataframe or the filename
"train" will accept both filenames and dataframes. It will detect them automatically and act accordingly. We will use the trainfile name here.

In [ ]:
model, cat_vocab_dict = deepauto.fit(train, target, keras_model_type=keras_model_type,
		project_name=project_name, keras_options=keras_options,  
		model_options=model_options, save_model_flag=True, use_my_model='',
		model_use_case='', verbose=0)

## Let's make predictions using the trained model

In [ ]:
predictions = deepauto.predict(model, project_name, test_dataset=testfile,
                                 keras_model_type=keras_model_type, 
                                 cat_vocab_dict=cat_vocab_dict)

In [ ]:
predictions[1]

In [ ]:
sub = pd.read_csv(subfile)
sub[target] = predictions[1]
print(sub.shape)
sub.head()

In [ ]:
sub.to_csv('submission.csv', index=False)